# VGG + DEEP CLUSTERING + BRAIN TUMOR

In [ ]:
# ! pip install torch torchvision numpy pydantic interpret faiss-cpu torchsummary matplotlib pandas seaborn scikit-learn scikit-image pytorch-lightning[extra] --no-cache-dir --ignore-installed

In [ ]:
import copy
import os
import pickle
import logging
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from data.brain_tumor_mri import BrainTumorMRI
from models.vgg_deep import DVGG
from pytorch_lightning import Trainer, seed_everything

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Timer
from pytorch_lightning.callbacks import RichProgressBar

from pytorch_lightning.loggers import CSVLogger
from torchvision import transforms
from torchvision.transforms import functional as F

from helpers.prediction import get_labels_from_prediction
from helpers.metrics import dmetrics_to_dataframe
from report.plots import plot_loss_curve, plot_acc_curve, plot_analysis, plot_clustering_metrics
from report import basic_report

seed_everything(42, workers=True)
pd.set_option('display.max_rows', None)

In [ ]:
BATCH_SIZE = 64
VAL_SIZE = 0.3
EPOCHS = 60
DATASET_RANDOM = 37

brain_tumor = BrainTumorMRI(val_size=VAL_SIZE, batch_size=BATCH_SIZE, random_state=DATASET_RANDOM, transformations=transforms.Compose(
            [
                transforms.Resize((255, 255)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ))

# MODEL

In [ ]:
model = DVGG(dataset=brain_tumor, num_classes=4, freeze=False)

timer = Timer()
progress_bar = RichProgressBar()
logger = CSVLogger(save_dir="./", name=f"experiments-1/{model.name}", flush_logs_every_n_steps=1)
checkpoint_callback = ModelCheckpoint(
    filename=f"{model.name}-" + "{epoch}-{train_loss:.4f}-{train_acc:.4f}-{val_loss:.4f}-{val_acc:.4f}",
    save_top_k=3, save_last=True, monitor="val_loss", mode="min", verbose=True
)

trainer = Trainer(
    accelerator="auto", 
    devices="auto",
    max_epochs=EPOCHS,
    logger=logger,
    callbacks=[timer, progress_bar, checkpoint_callback],
    log_every_n_steps=1,    
    # enable_progress_bar=False,
    # limit_train_batches=DATASET_RATIO or 1.0,
    # limit_val_batches=DATASET_RATIO or 1.0,
    # limit_test_batches=DATASET_RATIO or 1.0
)

## TRAINING

In [ ]:
trainer.fit(
    model,
    train_dataloaders=brain_tumor.train_dataloader,
    val_dataloaders=brain_tumor.val_dataloader
)

In [ ]:
with open(f"{trainer.logger.log_dir}/{model.name}-timing.dump", "wb") as f:
    pickle.dump(model.timers, f)

In [ ]:
with open(f"{trainer.logger.log_dir}/{model.name}-original-targets.dump", "wb") as f:
    pickle.dump(model.original_train_targets, f)

## TESTING

In [ ]:
batch_predictions = trainer.predict(model, dataloaders=brain_tumor.test_dataloader)
truth_labels, preds_labels = get_labels_from_prediction(batch_predictions)
with open(f"{trainer.logger.log_dir}/{model.name}-preds.dump", "wb") as f:
    pickle.dump([truth_labels, preds_labels], f)

## ANALYSIS

In [ ]:
train_metrics, valid_metrics, epoch_metrics, clustering_metrics = dmetrics_to_dataframe(f"{trainer.logger.log_dir}/metrics.csv", brain_tumor)

In [ ]:
plot_analysis(epoch_metrics, save_as=f"{trainer.logger.log_dir}/{model.name}-analysis.png")

In [ ]:
plot_clustering_metrics(clustering_metrics[1:], show=True, save_as=f"{trainer.logger.log_dir}/{model.name}-clustering.png")

In [ ]:
plot_loss_curve(epoch_metrics, show=False, save_as=f"{trainer.logger.log_dir}/{model.name}-loss.png")
plot_acc_curve(epoch_metrics, show=False, save_as=f"{trainer.logger.log_dir}/{model.name}-acc.png")
basic_report(truth_labels, preds_labels, labels=[0,1,2,3], label_names=["Glioma", "Meningioma", "Sem Tumor", "Pituitária"], show=False, 
             save_as=f"{trainer.logger.log_dir}/{model.name}-confusion-matrix.png")